In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [3]:
# -------------------------------
# Paths to dataset
# -------------------------------
train_dir = r"C:\Users\kotaa\Downloads\emotion_recognition_1\emotion_recognition\data\train"
test_dir  = r"C:\Users\kotaa\Downloads\emotion_recognition_1\emotion_recognition\data\test"

# -----

In [4]:
# -------------------------------
# Image Data Generators
# -------------------------------
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

img_height, img_width = 224, 224   # MobileNetV2 expects 224x224

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=64,
    class_mode="categorical"
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=64,
    class_mode="categorical"
)



Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [5]:
# -------------------------------
# Pretrained MobileNetV2 Model
# -------------------------------
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  # Freeze base model

# Add custom classifier on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation="relu"),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation="softmax")
])

In [6]:
# -------------------------------
# Compile Model
# -------------------------------
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [7]:
# -------------------------------
# Callbacks
# -------------------------------
checkpoint = ModelCheckpoint("emotion_mobilenetv2.h5", monitor="val_accuracy", save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)


In [9]:
# -------------------------------
# Train Model
# -------------------------------
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=13,
    callbacks=[checkpoint, early_stop]
)


Epoch 1/13
449/449 [==============================] - ETA: 0s - loss: 1.8271 - accuracy: 0.2709
Epoch 1: val_accuracy improved from -inf to 0.35093, saving model to emotion_mobilenetv2.h5
449/449 [==============================] - 430s 951ms/step - loss: 1.8271 - accuracy: 0.2709 - val_loss: 1.6646 - val_accuracy: 0.3509
Epoch 2/13
449/449 [==============================] - ETA: 0s - loss: 1.6804 - accuracy: 0.3360
Epoch 2: val_accuracy improved from 0.35093 to 0.38729, saving model to emotion_mobilenetv2.h5
449/449 [==============================] - 441s 982ms/step - loss: 1.6804 - accuracy: 0.3360 - val_loss: 1.6060 - val_accuracy: 0.3873
Epoch 3/13
449/449 [==============================] - ETA: 0s - loss: 1.6466 - accuracy: 0.3517
Epoch 3: val_accuracy improved from 0.38729 to 0.39691, saving model to emotion_mobilenetv2.h5
449/449 [==============================] - 418s 930ms/step - loss: 1.6466 - accuracy: 0.3517 - val_loss: 1.5696 - val_accuracy: 0.3969
Epoch 4/13
449/449 [=====

In [10]:
# -------------------------------
# Evaluate
# -------------------------------
loss, acc = model.evaluate(validation_generator)
print(f"✅ Test Accuracy: {acc*100:.2f}%")


113/113 [==============================] - 56s 491ms/step - loss: 1.4650 - accuracy: 0.4419
✅ Test Accuracy: 44.19%


In [11]:
model.save("final_emotion_mobilenetv2.h5")
print("🎉 Transfer Learning Model training completed and saved!")


🎉 Transfer Learning Model training completed and saved!
